In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-PSTRJPQO.fritz.box:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1737423746788)
SparkSession available as 'spark'


import org.apache.spark


In [ ]:
val sc = spark.SparkContext.getOrCreate()

In [2]:
val path_to_datasets = "../../../datasets/processed"

val path_to_tracks = path_to_datasets + "/tracks.csv"
val path_to_playlists = path_to_datasets + "/playlists.csv"
val path_to_track_in_playlists = path_to_datasets + "/tracks_in_playlist.csv"
val path_to_artists = path_to_datasets + "/artists.csv"

path_to_datasets: String = ../../../datasets/processed
path_to_tracks: String = ../../../datasets/processed/tracks.csv
path_to_playlists: String = ../../../datasets/processed/playlists.csv
path_to_track_in_playlists: String = ../../../datasets/processed/tracks_in_playlist.csv
path_to_artists: String = ../../../datasets/processed/artists.csv


In [3]:
object CsvParser {

  val noGenresListed = "(no genres listed)"
  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val quotes = "\""

  
  // (PID, playlist_name, num_followers)
  def parsePlayListLine(line: String): Option[(String, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt)
    } catch {
      case _: Exception => None
    }
  }

  // (track_uri, track_name, duration_ms, artist_uri, album_uri, album_name)
  def parseTrackLine(line: String): Option[(String, String, Int, String, String, String)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt, input(3).trim, input(4).trim, input(5).trim)
    } catch {
      case _: Exception => None
    }
  }

  // (artist_uri, artist_name)
  def parseArtistLine(line: String): Option[(String, String)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim)
    } catch {
      case _: Exception => None
    }
  }

  // (PID, track_uri, pos)
  def parseTrackInPlaylistLine(line: String): Option[(String, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt)
    } catch {
      case _: Exception => None
    }
  }
}

defined object CsvParser


In [4]:
val rddTracks = sc.textFile(path_to_tracks).
  flatMap(CsvParser.parseTrackLine)

val rddPlaylists = sc.textFile(path_to_playlists).
  flatMap(CsvParser.parsePlayListLine)

val rddTrackInPlaylists = sc.textFile(path_to_track_in_playlists).
  flatMap(CsvParser.parseTrackInPlaylistLine)

val rddArtists = sc.textFile(path_to_artists).
  flatMap(CsvParser.parseArtistLine)

rddTracks: org.apache.spark.rdd.RDD[(String, String, Int, String, String, String)] = MapPartitionsRDD[2] at flatMap at <console>:31
rddPlaylists: org.apache.spark.rdd.RDD[(String, String, Int)] = MapPartitionsRDD[5] at flatMap at <console>:34
rddTrackInPlaylists: org.apache.spark.rdd.RDD[(String, String, Int)] = MapPartitionsRDD[8] at flatMap at <console>:37
rddArtists: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[11] at flatMap at <console>:40


In [5]:
 
// For every song, find the most similar song, i.e., the one that appears most frequently in the same playlist.
// (Join-Join-Aggregate)

// auto-join of track_in_playlist 
// to get all song pairs in the same playlist

// RDD of (pid, trackUri)
val playlistTracks = rddTrackInPlaylists.map(x => (x._1, x._2))

val rddTrackPairs = playlistTracks.join(playlistTracks)
    .filter { case (_, (track1, track2)) => track1 != track2 }  // no auto-pair
    .map { case (_, (track1, track2)) => ((track1, track2), 1) }   

//Forma: ((track1, track2), 1)

playlistTracks: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[12] at map at <console>:32
rddTrackPairs: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[17] at map at <console>:36


In [6]:
// count the number of times each pair appears together
val cooccurencesCount = rddTrackPairs.reduceByKey(_ + _)

cooccurencesCount: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[18] at reduceByKey at <console>:26


In [7]:
// get the most common pair for each track
val mostCommonPair = cooccurencesCount
          .map { case ((track1, track2), count) => (track1, (track2, count)) }
          .reduceByKey { case ((track2A, count1), (track2B, count2)) => if (count1 > count2) (track2A, count1) else (track2B, count2) }

mostCommonPair: org.apache.spark.rdd.RDD[(String, (String, Int))] = ShuffledRDD[20] at reduceByKey at <console>:28


In [8]:
//mostCommonPair.collect()

res0: Array[(String, (String, Int))] = Array((spotify:track:1mjcyWQPFsoG1Cb6Gl33Tk,(spotify:track:3kAwGo8mtUPsrctroxYLku,2)), (spotify:track:7y7bmax41rKlTLOmgNvzKR,(spotify:track:4awpwf3TeFWOtLiswRbKfr,1)), (spotify:track:7inXu0Eaeg02VsM8kHNvzM,(spotify:track:3T7dNA7O8c3Axj5WyDNcH3,11)), (spotify:track:1WHCuLyhiISWVENR0qXZ51,(spotify:track:57TUYBa41jfW56U2U9652l,1)), (spotify:track:7ccnwVhaD3ITUQ2x8EkilA,(spotify:track:5xoUgPXbMNUmoHU0Enwtwq,2)), (spotify:track:5oUV6yWdDM0R9Q2CizRhIt,(spotify:track:3ZMv9EzGoteNi5Qnx0KpEO,5)), (spotify:track:7AUOZzM5P8UDuA0zga0PP8,(spotify:track:57yfmPoMfWljcEl3qI1ADp,2)), (spotify:track:4bGEfWw5uEAnvYuTbESsMa,(spotify:track:4CbKVDZkYKdv69I4bCaKUq,1)), (spotify:track:2m6Wm0nBUJdDfCggvpWAnV,(spotify:track:12TE7Vt592RcM1G3EaaZ0f,2)), (spotify:track:3CpoeW0...


In [9]:

// RDD of (track_uri, name)
val trackDetails = rddTracks.map(x => (x._1, x._2))

// join with track details
val firstResult = mostCommonPair.
        join(trackDetails).
        map { case (track1, ((track2, count), track1Name)) => (track1, track1Name, track2, count) }



trackDetails: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[21] at map at <console>:27
firstResult: org.apache.spark.rdd.RDD[(String, String, String, Int)] = MapPartitionsRDD[25] at map at <console>:32


In [10]:
val finalResult = firstResult
        .map { case (track1, track1Name, track2, count) => (track2, (track1, track1Name, count)) }
        .join(trackDetails)
        .map { case (track2, ((track1, track1Name, count), track2Name)) =>
          (track1, track1Name, track2, track2Name, count)
        }.collect()


finalResult: Array[(String, String, String, String, Int)] = Array((spotify:track:6SDJcDg0BzVOuUGT0WEYKP,We Wish You a Merry Christmas,spotify:track:7inXu0Eaeg02VsM8kHNvzM,Let It Snow! Let It Snow! Let It Snow!,2), (spotify:track:1abiHuQJw6UZyFkekr7DDY,It Feels Like Christmas,spotify:track:7inXu0Eaeg02VsM8kHNvzM,Let It Snow! Let It Snow! Let It Snow!,2), (spotify:track:6JzICCKjjIzQ5M2YqzLhWF,Winter Wonderland,spotify:track:7inXu0Eaeg02VsM8kHNvzM,Let It Snow! Let It Snow! Let It Snow!,2), (spotify:track:5EWxWuzdUI5S8l1KTfheGn,Jingle Bell Rock,spotify:track:7inXu0Eaeg02VsM8kHNvzM,Let It Snow! Let It Snow! Let It Snow!,2), (spotify:track:5OUN3NDrirM4SVqxCfN0Zn,The Nutcracker. Op. 71: Act II Tableau 3: Divertissement: f. Mother Gigogne and the clowns,spotify:track:7inXu0Eaeg02VsM8kHNvzM,Let ...
